In [27]:
## NLTK PREPROCESSING INPUT

## Imports

In [28]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import json
import numpy as np
import random
from collections.abc import MutableMapping


In [29]:
import collections.abc

# 👇️ add attributes to `collections` module
# before you import the package that causes the issue
collections.Mapping = collections.abc.Mapping
collections.MutableMapping = collections.abc.MutableMapping
collections.Iterable = collections.abc.Iterable
collections.MutableSet = collections.abc.MutableSet
collections.Callable = collections.abc.Callable



In [30]:
import tensorflow as tf


In [31]:
def tokenize_sentence(sentence):
    tokenize_word = word_tokenize(sentence)
    return tokenize_word


In [32]:

def bag_of_words(tokenize_sentence, all_words):
    bag = np.zeros(len(all_words), dtype = np.float32)

    for ind,word in enumerate(all_words):
        if word in tokenize_sentence:
            bag[ind] = 1.0
    return bag
       
    

In [33]:
def stem_sentence(sentence,stemmer= PorterStemmer()):
    stemmed_sentence = []
    ignore_words = ["!", "?","," , "."]
    for word in sentence:
        if word not in ignore_words:
            word = word.lower()
            stemmed_word = stemmer.stem(word)
            stemmed_sentence.append(stemmed_word)
    return stemmed_sentence
        

In [34]:
all_words = []
tags = []
xy = []
vocabulary = []


with open ("intents.json", 'r') as file:
    intents = json.load(file)
for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    sentences = intent["patterns"]
    for items in sentences:
        
        w = tokenize_sentence(items)
        vocabulary.append(w)
        w = stem_sentence(w)
        xy.append((w,tag))
        all_words.extend(w)


ignore_words = ["!", "?","," , "."]
set_of_word = [i.lower() for word in vocabulary for i  in word if i not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))
train_x = []
train_y = []


with open ("tags.json", 'w') as file:
    json.dump(tags, file)

with open ("all_words.json", 'w') as file:
    json.dump(all_words, file)


In [35]:
print(set_of_word)

['hi', 'there', 'hello', 'hey', 'good', 'morning', 'i', 'want', 'to', 'schedule', 'a', 'cleaning', 'service', 'can', 'i', 'book', 'a', 'cleaning', 'appointment', 'how', 'can', 'i', 'set', 'up', 'a', 'cleaning', 'appointment', 'schedule', 'a', 'cleaning', 'for', 'tomorrow', 'what', 'janitorial/cleaning', 'services', 'do', 'you', 'offer', 'tell', 'me', 'about', 'your', 'cleaning', 'services', 'what', 'secure', 'and', 'logistic', 'services', 'do', 'you', 'provide', 'tell', 'me', 'about', 'your', 'executive', 'car', 'hire', 'service', 'what', 'business', 'services', 'do', 'you', 'offer', 'do', 'you', 'provide', 'recruitment', 'and', 'training', 'solutions', 'what', 'industries', 'do', 'you', 'specialize', 'in', 'how', 'much', 'does', 'your', 'janitorial', 'cleaning', 'service', 'cost', 'what', 'are', 'the', 'rates', 'for', 'home', 'cleaning', 'tell', 'me', 'about', 'your', 'post-construction', 'cleaning', 'pricing', 'how', 'much', 'does', 'outsourcing', 'cost', 'what', 'are', 'the', 'fees'

In [36]:
len(tags)

9

In [37]:
for (sequence, tag )in xy:
    bag = bag_of_words(sequence, all_words)
    train_x.append(bag)

    label = tags.index(tag)
    train_y.append(label)

In [38]:
#trainy one hot encoding
hot_output = []
output_code = [0 for i in range(len(tags))]
for tag_no in train_y:
    output_tag = output_code[:]
    output_tag[tag_no] = 1
    hot_output.append(output_tag)


In [39]:
train_x = np.array(train_x)
train_y = np.array(hot_output)

In [40]:
train_x.shape

(47, 121)

In [41]:
model = tf.keras.Sequential()

In [42]:
model.add(tf.keras.layers.Dense(40, activation = "relu", input_shape =(None, train_x.shape[0], train_x.shape[1]) ))
model.add(tf.keras.layers.Dense(35,activation = "relu"))
model.add(tf.keras.layers.Dense(len(tags), activation = "softmax"))

In [43]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [44]:
model.fit(train_x, train_y, epochs = 40, batch_size = 20)

Epoch 1/40


3/3 [==============================] - 2s 5ms/step - loss: 2.2000 - accuracy: 0.1489
Epoch 2/40
3/3 [==============================] - 0s 6ms/step - loss: 2.1561 - accuracy: 0.1489
Epoch 3/40
3/3 [==============================] - 0s 6ms/step - loss: 2.1188 - accuracy: 0.1915
Epoch 4/40
3/3 [==============================] - 0s 4ms/step - loss: 2.0832 - accuracy: 0.2128
Epoch 5/40
3/3 [==============================] - 0s 7ms/step - loss: 2.0523 - accuracy: 0.2553
Epoch 6/40
3/3 [==============================] - 0s 6ms/step - loss: 2.0181 - accuracy: 0.2979
Epoch 7/40
3/3 [==============================] - 0s 5ms/step - loss: 1.9856 - accuracy: 0.3830
Epoch 8/40
3/3 [==============================] - 0s 5ms/step - loss: 1.9537 - accuracy: 0.3830
Epoch 9/40
3/3 [==============================] - 0s 3ms/step - loss: 1.9193 - accuracy: 0.4043
Epoch 10/40
3/3 [==============================] - 0s 5ms/step - loss: 1.8861 - accuracy: 0.4681
Epoch 11/40
3/3 [==============================] -

In [45]:
model.save("model1")

INFO:tensorflow:Assets written to: model1\assets


INFO:tensorflow:Assets written to: model1\assets
